1 Implement a Kalman filter to predict and update the state of an object given its measurements

In [1]:
import numpy as np

class KalmanFilter1D:
    def __init__(self, dt, process_var, meas_var):
        """
        dt: time step
        process_var: variance of the process noise (model uncertainty)
        meas_var: variance of the measurement noise
        """
        # State vector [position, velocity]
        self.x = np.array([[0], [0]])  # Initial state

        # State transition matrix
        self.F = np.array([[1, dt],
                           [0, 1]])

        # Control input matrix (none in this example)
        self.B = np.array([[0],
                           [0]])

        # Measurement matrix (we only measure position)
        self.H = np.array([[1, 0]])

        # Process noise covariance
        self.Q = process_var * np.array([[dt**4/4, dt**3/2],
                                         [dt**3/2, dt**2]])

        # Measurement noise covariance
        self.R = np.array([[meas_var]])

        # Error covariance matrix (initial uncertainty)
        self.P = np.eye(2)

    def predict(self):
        # Predict state
        self.x = self.F @ self.x

        # Predict error covariance
        self.P = self.F @ self.P @ self.F.T + self.Q

    def update(self, z):
        """
        z: measurement (position)
        """
        # Measurement residual
        y = z - self.H @ self.x

        # Residual covariance
        S = self.H @ self.P @ self.H.T + self.R

        # Kalman gain
        K = self.P @ self.H.T @ np.linalg.inv(S)

        # Update state estimate
        self.x = self.x + K @ y

        # Update error covariance
        I = np.eye(self.P.shape[0])
        self.P = (I - K @ self.H) @ self.P

    def get_state(self):
        return self.x.flatten()


# Example usage
if __name__ == "__main__":
    dt = 1.0            # 1 second time step
    process_var = 1e-4  # small process noise
    meas_var = 0.1      # measurement noise variance

    kf = KalmanFilter1D(dt, process_var, meas_var)

    # Simulated noisy position measurements
    measurements = [1.0, 2.0, 3.0, 2.5, 3.5, 4.0, 5.0]

    for z in measurements:
        kf.predict()
        kf.update(np.array([[z]]))
        pos, vel = kf.get_state()
        print(f"Predicted Position: {pos:.2f}, Velocity: {vel:.2f}")

Predicted Position: 0.95, Velocity: 0.48
Predicted Position: 1.93, Velocity: 0.88
Predicted Position: 2.96, Velocity: 0.96
Predicted Position: 2.97, Velocity: 0.57
Predicted Position: 3.52, Velocity: 0.56
Predicted Position: 4.04, Velocity: 0.55
Predicted Position: 4.78, Velocity: 0.60


2 Write a function to normalize an image array such that pixel values are scaled between 0 and 1C

In [2]:
import numpy as np

def normalize_image(image):
    """
    Normalize a numpy image array to have pixel values between 0 and 1.

    Parameters:
        image (np.ndarray): Input image array (can be grayscale or color).

    Returns:
        np.ndarray: Normalized image array with values in [0, 1].
    """
    image = image.astype(np.float32)  # Convert to float for precision
    min_val = np.min(image)
    max_val = np.max(image)

    # Avoid division by zero if image is constant
    if max_val - min_val == 0:
        return np.zeros_like(image)

    normalized = (image - min_val) / (max_val - min_val)
    return normalized

In [3]:
import cv2

img = cv2.imread('your_image.jpg')  # Read image as NumPy array
normalized_img = normalize_image(img)
print(normalized_img.min(), normalized_img.max())  # Should print 0.0 1.0


AttributeError: 'NoneType' object has no attribute 'astype'

3 Create a function to generate dummy object detection data with confidence scores and bounding boxes.
Filter the detections based on a confidence thresholdC

In [4]:
import numpy as np

def generate_dummy_detections(num_detections=10):
    """
    Generate dummy object detection data.

    Returns:
        detections (list of dict): Each dict has 'bbox' and 'confidence'.
            bbox format: [x_min, y_min, x_max, y_max]
            confidence: float between 0 and 1
    """
    detections = []
    for _ in range(num_detections):
        x_min = np.random.uniform(0, 400)
        y_min = np.random.uniform(0, 400)
        width = np.random.uniform(20, 100)
        height = np.random.uniform(20, 100)
        x_max = x_min + width
        y_max = y_min + height
        confidence = np.random.uniform(0, 1)
        detections.append({
            'bbox': [x_min, y_min, x_max, y_max],
            'confidence': confidence
        })
    return detections

def filter_detections_by_confidence(detections, threshold=0.5):
    """
    Filter detections based on confidence threshold.

    Args:
        detections (list of dict): Detections with 'bbox' and 'confidence'
        threshold (float): Confidence threshold between 0 and 1

    Returns:
        filtered_detections (list of dict): Only detections with confidence >= threshold
    """
    return [d for d in detections if d['confidence'] >= threshold]


# Example usage
if __name__ == "__main__":
    detections = generate_dummy_detections(15)
    print("All Detections:")
    for d in detections:
        print(f"bbox: {d['bbox']}, confidence: {d['confidence']:.2f}")

    filtered = filter_detections_by_confidence(detections, threshold=0.6)
    print("\nFiltered Detections (conf >= 0.6):")
    for d in filtered:
        print(f"bbox: {d['bbox']}, confidence: {d['confidence']:.2f}")

All Detections:
bbox: [228.81989863039277, 144.00134134937898, 256.948880168382, 174.6094224473559], confidence: 0.86
bbox: [336.6666621646263, 258.44525278884373, 374.7715107806456, 291.8616363352743], confidence: 0.25
bbox: [40.41169262425828, 333.1516735944229, 69.88358096336853, 410.6254284445246], confidence: 0.84
bbox: [151.31082465554923, 334.04170488077705, 183.65904108121794, 373.68787007774154], confidence: 0.93
bbox: [370.2999719026903, 110.30342955083596, 405.93787405929316, 138.49587477340359], confidence: 0.02
bbox: [150.02835950927098, 192.2501269821117, 224.06940850079127, 269.67376265216336], confidence: 0.71
bbox: [258.3191419919244, 5.754111959223485, 323.93015863394885, 97.51778073244888], confidence: 0.10
bbox: [291.6545523460877, 18.353015486012936, 336.01922288727906, 96.65804591868078], confidence: 0.34
bbox: [181.55530284910478, 309.99371932046984, 280.16758567676175, 357.6055595040455], confidence: 0.55
bbox: [393.92869248064204, 121.43909676275601, 450.368465

4 Write a function that takes a list of YOLO detections and extracts a random 128-dimensional feature vector
for each detectionC

In [5]:
import numpy as np

def extract_random_features(detections):
    """
    For each detection, generate a random 128-dimensional feature vector.

    Args:
        detections (list): List of detections (any format, e.g., dicts or lists)

    Returns:
        features (list of np.ndarray): Each is a 128-dim vector for a detection
    """
    features = []
    for _ in detections:
        # Generate random vector with values between 0 and 1
        feature_vector = np.random.rand(128).astype(np.float32)
        features.append(feature_vector)
    return features

# Example usage
if __name__ == "__main__":
    dummy_detections = [
        {'bbox': [100, 150, 200, 250], 'confidence': 0.9, 'class': 'person'},
        {'bbox': [300, 350, 400, 450], 'confidence': 0.8, 'class': 'car'},
    ]

    features = extract_random_features(dummy_detections)
    for i, fvec in enumerate(features):
        print(f"Detection {i} feature shape: {fvec.shape}")
        print(fvec[:5])  # print first 5 elements of the feature vector

Detection 0 feature shape: (128,)
[0.42202422 0.6101005  0.5191031  0.92109203 0.8043286 ]
Detection 1 feature shape: (128,)
[0.79414254 0.0805231  0.1148926  0.7859115  0.70892394]


5  Write a function to re-identify objects by matching feature vectors based on Euclidean distance

In [6]:
import numpy as np

def reidentify_objects(features_old, features_new, distance_threshold=0.5):
    """
    Match new features to old features based on Euclidean distance for re-identification.

    Args:
        features_old (list of np.ndarray): Feature vectors of previously tracked objects
        features_new (list of np.ndarray): Feature vectors of current detections
        distance_threshold (float): Max distance to consider a match

    Returns:
        matches (list of tuples): (index_old, index_new) pairs of matched features
        unmatched_new (list): indices of new features with no match
        unmatched_old (list): indices of old features with no match
    """
    matches = []
    unmatched_new = set(range(len(features_new)))
    unmatched_old = set(range(len(features_old)))

    # Compute distance matrix: rows=old, cols=new
    dist_matrix = np.zeros((len(features_old), len(features_new)))
    for i, f_old in enumerate(features_old):
        for j, f_new in enumerate(features_new):
            dist = np.linalg.norm(f_old - f_new)
            dist_matrix[i, j] = dist

    # Greedy matching based on minimum distance
    while True:
        if dist_matrix.size == 0:
            break

        min_idx = np.unravel_index(np.argmin(dist_matrix), dist_matrix.shape)
        min_dist = dist_matrix[min_idx]

        if min_dist > distance_threshold:
            break  # No more matches within threshold

        i_old, j_new = min_idx
        matches.append((i_old, j_new))

        # Remove matched rows and columns by setting large distance
        dist_matrix[i_old, :] = np.inf
        dist_matrix[:, j_new] = np.inf

        unmatched_old.discard(i_old)
        unmatched_new.discard(j_new)

    return matches, list(unmatched_new), list(unmatched_old)


# Example usage
if __name__ == "__main__":
    # Create dummy old and new feature vectors (128D)
    features_old = [np.random.rand(128) for _ in range(3)]
    features_new = [np.random.rand(128) for _ in range(4)]

    matches, unmatched_new, unmatched_old = reidentify_objects(features_old, features_new, distance_threshold=2.0)

    print("Matches (old_idx -> new_idx):", matches)
    print("Unmatched new detections:", unmatched_new)
    print("Unmatched old tracks:", unmatched_old)

Matches (old_idx -> new_idx): []
Unmatched new detections: [0, 1, 2, 3]
Unmatched old tracks: [0, 1, 2]


6 Write a function to track object positions using YOLO detections and a Kalman FilterC

In [7]:
import numpy as np

class KalmanFilter2D:
    def __init__(self, dt=1.0, process_var=1e-4, meas_var=0.1):
        """
        State: [x, y, vx, vy]^T
        """
        self.dt = dt
        self.x = np.zeros((4, 1))  # initial state: position & velocity

        # State transition matrix
        self.F = np.array([[1, 0, dt, 0],
                           [0, 1, 0, dt],
                           [0, 0, 1, 0 ],
                           [0, 0, 0, 1 ]])

        # Measurement matrix (we measure position only)
        self.H = np.array([[1, 0, 0, 0],
                           [0, 1, 0, 0]])

        # Process noise covariance
        q = process_var
        self.Q = q * np.array([[dt**4/4, 0, dt**3/2, 0],
                               [0, dt**4/4, 0, dt**3/2],
                               [dt**3/2, 0, dt**2, 0],
                               [0, dt**3/2, 0, dt**2]])

        # Measurement noise covariance
        self.R = meas_var * np.eye(2)

        # Initial error covariance
        self.P = np.eye(4)

    def predict(self):
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q

    def update(self, z):
        """
        z: measurement vector (2x1) - position [x, y]
        """
        y = z - self.H @ self.x  # measurement residual
        S = self.H @ self.P @ self.H.T + self.R  # residual covariance
        K = self.P @ self.H.T @ np.linalg.inv(S)  # Kalman gain
        self.x = self.x + K @ y
        I = np.eye(self.P.shape[0])
        self.P = (I - K @ self.H) @ self.P

    def get_position(self):
        return self.x[:2].flatten()

In [8]:
def track_objects_with_kalman(yolo_detections, kf):
    """
    Args:
        yolo_detections (list of dict): each with 'bbox': [x_min, y_min, x_max, y_max]
        kf (KalmanFilter2D): Kalman filter instance for tracking one object

    Returns:
        predicted_pos (np.ndarray): predicted position after update
    """
    # Predict step
    kf.predict()

    if len(yolo_detections) == 0:
        # No detections, return predicted position only
        return kf.get_position()

    # For simplicity, take first detection's bbox center as measurement
    bbox = yolo_detections[0]['bbox']
    x_center = (bbox[0] + bbox[2]) / 2
    y_center = (bbox[1] + bbox[3]) / 2
    measurement = np.array([[x_center], [y_center]])

    # Update step with measurement
    kf.update(measurement)

    return kf.get_position()

In [9]:
if __name__ == "__main__":
    kf = KalmanFilter2D()

    # Simulated YOLO detections for 5 frames
    frames_detections = [
        [{'bbox': [100, 50, 140, 90]}],
        [{'bbox': [102, 52, 142, 92]}],
        [{'bbox': [105, 55, 145, 95]}],
        [],  # missed detection (e.g., occlusion)
        [{'bbox': [110, 60, 150, 100]}],
    ]

    for i, detections in enumerate(frames_detections):
        pos = track_objects_with_kalman(detections, kf)
        print(f"Frame {i+1}: Tracked position: {pos}")

Frame 1: Tracked position: [114.28578231  66.66670635]
Frame 2: Tracked position: [128.0698046   75.43839107]
Frame 3: Tracked position: [130.65980433  78.13136465]
Frame 4: Tracked position: [142.18533681  85.76804471]
Frame 5: Tracked position: [134.64493269  82.62597054]


7 Implement a simple Kalman Filter to track an object's position in a 2D space (simulate the object's
movement with random noise)

In [10]:
import numpy as np
import matplotlib.pyplot as plt

class KalmanFilter2D:
    def __init__(self, dt=1.0, process_var=1e-4, meas_var=0.1):
        # State vector [x, y, vx, vy]
        self.dt = dt
        self.x = np.zeros((4, 1))  # initial state

        # State transition matrix
        self.F = np.array([[1, 0, dt, 0],
                           [0, 1, 0, dt],
                           [0, 0, 1, 0],
                           [0, 0, 0, 1]])

        # Measurement matrix (we measure position only)
        self.H = np.array([[1, 0, 0, 0],
                           [0, 1, 0, 0]])

        # Process noise covariance
        q = process_var
        self.Q = q * np.array([[dt**4/4, 0, dt**3/2, 0],
                               [0, dt**4/4, 0, dt**3/2],
                               [dt**3/2, 0, dt**2, 0],
                               [0, dt**3/2, 0, dt**2]])

        # Measurement noise covariance
        self.R = meas_var * np.eye(2)

        # Initial error covariance
        self.P = np.eye(4)

    def predict(self):
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q

    def update(self, z):
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        I = np.eye(self.P.shape[0])
        self.P = (I - K @ self.H) @ self.P

    def get_position(self):
        return self.x[:2].flatten()


def simulate_object_motion(num_steps=50, dt=1.0):
    """
    Simulate true object movement with constant velocity plus noise,
    and generate noisy measurements.
    """
    # True initial state [x, y, vx, vy]
    true_state = np.array([[0], [0], [1], [0.5]])

    # Matrices same as Kalman Filter's
    F = np.array([[1, 0, dt, 0],
                  [0, 1, 0, dt],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])

    process_noise_std = 0.05
    measurement_noise_std = 0.5

    true_states = []
    measurements = []

    state = true_state.copy()

    for _ in range(num_steps):

SyntaxError: incomplete input (<ipython-input-10-d748bcf03c67>, line 71)